In [1]:
96 / 16

6.0

In [ ]:
import json

from sqlalchemy import create_engine, text
import pandas as pd

# 数据库配置
import pickle
with open('./datasets/sql_token.pkl', 'rb') as f:
    DB_URI = pickle.load(f)

engine = create_engine(DB_URI)
DATA_JSON_PATH = './datasets/data.json'

# 加载分类数据
with open(DATA_JSON_PATH) as f:
    group_map = json.load(f)


In [ ]:
for group_idx, (group_name, group_list) in enumerate(group_map.items()):
    for group_data in group_list:
        if group_name == 'stock1' and group_data['industry_code'] in ['270000', '480000']:
            print(group_name, group_data, )
        # break
    # break

In [ ]:
df = {}
for i in range(len(group_map['stock1'])):
    if group_map['stock1'][i]['industry_code'] == '270000':
        print(group_map['stock1'][i])
        df['stock-270000'] = group_map['stock1'][i]['code_list']
df

In [ ]:
import pickle
with open('./datasets/benchmark.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
import pickle
with open('./datasets/benchmark.pkl', 'rb') as f:
    group = pickle.load(f)
group

In [ ]:
pred_value = 'accnav'
sql = text(f"SELECT fund_code, date, nav, accnav, adj_nav FROM b_fund_nav_details_new WHERE fund_code IN :codes AND date BETWEEN :start AND :end ORDER BY date")
df = pd.read_sql_query(
    sql.bindparams(codes=tuple(['013152']), start='2020-07-13', end='2025-03-8'),
    engine
)
df

In [ ]:
df_stamp = pd.to_datetime(df['date'])
df_stamp[1].year, df_stamp[1].month, df_stamp[1].day, df_stamp[1].weekday()

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_excel('./datasets/lottery/pl3_desc.xls', header=1)
timestamps = pd.to_datetime(df['开奖日期'])
timestamps = np.array([[ts.year, ts.month, ts.day, ts.weekday()] for ts in timestamps])
x = np.array(df['号']).reshape(-1, 1)
# timestamps
x = np.concatenate((timestamps, x), axis=-1)
df

In [ ]:
from modules.pretrain_timer import Timer
import torch
from utils.exp_config import get_config
config = get_config()
config.patch_len = 96
config.d_model = 1024
config.d_ff = 2048
config.e_layers = 8
config.n_heads = 8
config.dropout = 0.10
config.factor = 1
config.output_attention = 1
config.activation = 'gelu'
config.ckpt_path = 'Timer_forecast_1.0.ckpt'
backbone = Timer(config)
ckpt_path = 'Timer_forecast_1.0.ckpt'
sd = torch.load(ckpt_path, weights_only=False, map_location="cpu")["state_dict"]
sd = {k[6:]: v for k, v in sd.items()}
backbone.load_state_dict(sd, strict=True)

In [1]:
from utils.exp_logger import Logger
from utils.exp_metrics_plotter import MetricsPlotter
from run_train import get_experiment_name
from utils.utils import set_settings
from a_data_center import DataModule
# Experiment Settings, logger, plotter
from utils.exp_config import get_config
config = get_config()
config.multi_dataset = True
set_settings(config)
log_filename, exper_detail = get_experiment_name(config)
plotter = MetricsPlotter(log_filename, config)
log = Logger(log_filename, exper_detail, plotter, config)
datamodule = DataModule(config)
# model = Model(datamodule, config)

|2025-05-14 13:39:14| {
     'ablation': 0, 'att_method': self, 'bs': 32
     'classification': False, 'continue_train': False, 'dataset': financial
     'debug': False, 'decay': 0.0001, 'density': 0.7
     'device': cuda, 'dis_method': cosine, 'epochs': 200
     'eval_set': True, 'ffn_method': moe, 'fft': False
     'hyper_search': False, 'idx': 0, 'log': <utils.exp_logger.Logger object at 0x738009311f70>
     'logger': None, 'loss_func': MSELoss, 'lr': 0.001
     'model': ours, 'monitor_metric': MAE, 'multi_dataset': True
     'norm_method': rms, 'num_layers': 4, 'optim': AdamW
     'path': ./datasets/, 'patience': 30, 'pred_len': 10
     'rank': 64, 'record': True, 'retrain': True
     'revin': False, 'rounds': 1, 'scaler_method': stander
     'seed': 0, 'seq_len': 48, 'shuffle': False
     'train_size': 500, 'try_exp': 1, 'ts_var': 0
     'use_train_size': False, 'verbose': 5
}
./datasets/financial/560100.pkl
(761, 8) (761,)
./datasets/financial/560100.pkl
(761, 8) (761,)
./dataset

/home/rtx4090/code/python/TimeSeries/modules/load_data/get_financial.py:133: RuntimeWarning: invalid value encountered in divide
  all_train_x[:, -i] = (all_train_x[:, -i] - now_mean) / now_std
/home/rtx4090/code/python/TimeSeries/modules/load_data/get_financial.py:134: RuntimeWarning: invalid value encountered in divide
  all_valid_x[:, -i] = (all_valid_x[:, -i] - now_mean) / now_std
/home/rtx4090/code/python/TimeSeries/modules/load_data/get_financial.py:135: RuntimeWarning: invalid value encountered in divide
  all_test_x[:, -i] = (all_test_x[:, -i] - now_mean) / now_std


In [36]:
import torch
import torch.nn as nn

# 假设输入
bs, seq_len, channels, dim = 16, 48, 33, 64
x_enc = torch.randn(bs, seq_len, channels, dim)

# 定义 attention 层（无 batch_first 参数）
attn_channel = nn.MultiheadAttention(embed_dim=dim, num_heads=8)  # expects (seq_len, batch, dim)
attn_time = nn.MultiheadAttention(embed_dim=dim, num_heads=8)

# ===== 1. 跨通道 attention =====
# 原始 x_enc: (bs, 48, 33, 64)
# 调整为 (33, bs*48, 64)
x_enc_reshaped = x_enc.permute(2, 0, 1, 3).reshape(channels, bs * seq_len, dim)

# 注意力：通道之间的 self-attention
x_channel_attn, _ = attn_channel(x_enc_reshaped, x_enc_reshaped, x_enc_reshaped)  # (33, bs*48, 64)

# 还原为 (bs, 48, 33, 64)
x_channel_attn = x_channel_attn.reshape(channels, bs, seq_len, dim).permute(1, 2, 0, 3)

# ===== 2. 跨时间 attention =====
# 调整为 (48, bs*33, 64)
x_time_input = x_channel_attn.permute(1, 0, 2, 3).reshape(seq_len, bs * channels, dim)

# 注意力：时间步之间的 self-attention
x_time_attn, _ = attn_time(x_time_input, x_time_input, x_time_input)  # (48, bs*33, 64)

# 还原为 (bs, 48, 33, 64)
x_time_attn = x_time_attn.reshape(seq_len, bs, channels, dim).permute(1, 0, 2, 3)

# 最终输出
print(x_time_attn.shape)  # torch.Size([16, 48, 33, 64])


torch.Size([16, 48, 33, 64])


In [26]:
from modules.backbone import Backbone
from run_train import *

from utils.exp_config import get_config
config = get_config()
# datamodule = DataModule(config)
# model = Model(datamodule, config)
model = Backbone(3, config)

In [27]:
bs, seq_len, channels, dim = 1, 48, 33, 3
random_inputs = torch.rand(bs, seq_len, channels, dim)
y = model(random_inputs, None, None)
# [1, 48, 32, 3]

In [28]:
bs, seq_len, channels, dim = 1, 48, 1, 3
random_inputs = torch.rand(bs, seq_len, channels, dim)
y = model(random_inputs, None, None)
# [1, 48, 32, 3]

In [17]:
bs, seq_len, channels, dim = 1, 48, 16, 3
random_inputs = torch.rand(bs, seq_len, channels, dim)
y = model(random_inputs, None, None)
# [1, 48, 32, 3]

In [3]:
import pandas as pd 
df = pd.read_csv('datasets/weather/weather.csv')
df

,date,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m�),PAR (�mol/m�/s),max. PAR (�mol/m�/s),Tlog (degC),OT
0,2020-01-01 00:10:00,1008.89,0.71,273.18,-1.33,86.1,6.43,5.54,0.89,3.42,...,1.02,1.60,224.3,0.0,0.0,0.0,0.0,0.0,11.45,428.1
1,2020-01-01 00:20:00,1008.76,0.75,273.22,-1.44,85.2,6.45,5.49,0.95,3.39,...,0.43,0.84,206.8,0.0,0.0,0.0,0.0,0.0,11.51,428.0
2,2020-01-01 00:30:00,1008.66,0.73,273.21,-1.48,85.1,6.44,5.48,0.96,3.39,...,0.61,1.48,197.1,0.0,0.0,0.0,0.0,0.0,11.60,427.6
3,2020-01-01 00:40:00,1008.64,0.37,272.86,-1.64,86.3,6.27,5.41,0.86,3.35,...,1.11,1.48,206.4,0.0,0.0,0.0,0.0,0.0,11.70,430.0
4,2020-01-01 00:50:00,1008.61,0.33,272.82,-1.50,87.4,6.26,5.47,0.79,3.38,...,0.49,1.40,209.6,0.0,0.0,0.0,0.0,0.0,11.81,432.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52691,2020-12-31 23:20:00,978.32,2.28,277.16,-0.80,80.0,7.20,5.76,1.44,3.67,...,0.73,1.40,180.6,0.0,0.0,0.0,0.0,0.0,13.40,433.0
52692,2020-12-31 23:30:00,978.30,2.13,277.01,-0.43,83.1,7.12,5.92,1.20,3.77,...,0.43,0.82,174.0,0.0,0.0,0.0,0.0,0.0,13.42,439.6
52693,2020-12-31 23:40:00,978.26,1.99,276.88,-0.71,82.2,7.05,5.80,1.26,3.69,...,0.38,0.76,248.9,0.0,0.0,0.0,0.0,0.0,13.45,435.2
52694,2020-12-31 23:50:00,978.26,2.07,276.95,-0.77,81.4,7.09,5.77,1.32,3.68,...,0.57,1.07,196.6,0.0,0.0,0.0,0.0,0.0,13.47,433.9
